# Multimodal AI Pipeline with Ray Data

**Time to complete**: 30 min | **Difficulty**: Advanced | **Prerequisites**: ML experience, understanding of computer vision and NLP

## What You'll Build

Create a cutting-edge multimodal AI system that processes images and text together - like how humans understand memes, social media posts, or product listings that combine visual and textual information.

## Table of Contents

1. [Multimodal Data Creation](#step-1-creating-multimodal-data) (7 min)
2. [Image Processing](#step-2-image-feature-extraction) (8 min)
3. [Text Processing](#step-3-text-feature-extraction) (8 min)
4. [Multimodal Fusion](#step-4-cross-modal-fusion) (7 min)

## Learning Objectives

By completing this tutorial, you'll understand:

- **Why multimodal AI matters**: How combining different data types creates smarter AI systems
- **Ray Data's multimodal power**: Process images, text, and other data types in unified pipelines
- **Real-world applications**: How companies like Instagram, Pinterest, and TikTok analyze content
- **Fusion techniques**: Combine information from different modalities for better predictions

## Overview

**The Challenge**: Traditional AI processes one type of data at a time (just images OR just text). But real-world content is multimodal - think Instagram posts with images and captions, or product listings with photos and descriptions.

**The Solution**: Ray Data enables processing multiple data types in parallel, then combining their insights for more accurate and comprehensive AI understanding.

**Real-world Impact**:
- **Social Media**: Analyze posts with images and captions for content moderation
- **E-commerce**: Match product images with descriptions for better search
- **Entertainment**: Understand video content with both visual and audio cues
- **Healthcare**: Combine medical images with patient text records

---

## Prerequisites Checklist

Before starting, ensure you have:
- [ ] Understanding of deep learning concepts
- [ ] Familiarity with computer vision and NLP basics
- [ ] Experience with PyTorch or similar ML frameworks
- [ ] GPU access recommended (but not required)

## Quick Start (3 minutes)

Want to see multimodal processing immediately?

In [ ]:
import ray

# Create sample multimodal data
data = [
    {"image_path": "sample.jpg", "caption": "Beautiful sunset over mountains"},
    {"image_path": "sample2.jpg", "caption": "Delicious pizza with cheese"}
]
ds = ray.data.from_items(data)
print(f" Created multimodal dataset with {ds.count()} items")

To run this template, you will need the following packages:

In [ ]:
%%bash
pip install ray[data] torch torchvision transformers numpy pillow

---

## Step 1: Creating Multimodal Data
*⏱ Time: 7 minutes*

### What We're Doing
We'll create a realistic multimodal dataset that combines images with text descriptions - similar to social media posts, product listings, or news articles with photos.

### Why Multimodal Data Matters
- **Richer Understanding**: Combining text and images provides more context than either alone
- **Real-World Relevance**: Most real-world data is multimodal (posts, products, documents)
- **Better AI**: Multimodal models consistently outperform single-modal approaches

In [ ]:
import ray
import torch
import numpy as np
from PIL import Image
import time

# Initialize Ray for distributed multimodal processing
print(" Initializing Ray for multimodal AI...")
start_time = time.time()
ray.init()
init_time = time.time() - start_time

print(f" Ray cluster ready in {init_time:.2f} seconds")
print(f" Available resources: {ray.cluster_resources()}")

# Check for GPU availability - crucial for multimodal models
gpu_count = ray.cluster_resources().get('GPU', 0)
if gpu_count > 0:
    print(f" GPU acceleration available: {gpu_count} GPUs detected")
    device = "cuda"
else:
    print(" Using CPU processing (GPU highly recommended for multimodal AI)")
    device = "cpu"

def create_multimodal_dataset():
    """Create realistic multimodal data combining images and text."""
    print(" Creating multimodal dataset...")
    
    # Sample image descriptions that might appear with photos
    image_captions = [
        "Beautiful sunset over the mountains with orange and pink sky",
        "Delicious homemade pizza with cheese and pepperoni",
        "Cute golden retriever playing in the park with a ball",
        "Modern city skyline at night with illuminated buildings",
        "Fresh vegetables and fruits at the farmer's market",
        "Cozy coffee shop interior with wooden tables and warm lighting",
        "Snow-covered forest path during winter season",
        "Colorful flowers blooming in a spring garden",
        "Ocean waves crashing against rocky cliffs",
        "Vintage car parked on a quiet street"
    ]
    
    # Create multimodal data pairs
    multimodal_data = []
    
    for i in range(1000):  # Create 1000 multimodal samples
        # Create a simple synthetic image (colored square)
        # In real applications, you'd load actual images
        color = np.random.randint(0, 256, 3)  # Random RGB color
        image = np.full((64, 64, 3), color, dtype=np.uint8)  # 64x64 colored square
        
        # Select a caption and add some variation
        base_caption = np.random.choice(image_captions)
        caption = f"{base_caption} (sample {i+1})"
        
        multimodal_data.append({
            'item_id': f'item_{i:04d}',
            'image': image,
            'text': caption,
            'category': np.random.choice(['nature', 'food', 'urban', 'animals']),
            'length': len(caption)
        })
    
    return ray.data.from_items(multimodal_data)

# Create our multimodal dataset
multimodal_dataset = create_multimodal_dataset()

# Display dataset information
print(f" Created multimodal dataset: {multimodal_dataset.count():,} items")
print(f" Schema: {multimodal_dataset.schema()}")

# Show sample multimodal data
print("\n Sample multimodal data:")
samples = multimodal_dataset.take(3)
for i, sample in enumerate(samples):
    print(f"  {i+1}. {sample['item_id']}: '{sample['text'][:50]}...' + {sample['image'].shape} image")

** What just happened?**
- Created 1,000 multimodal samples with both images and text
- Each sample has a synthetic image and descriptive text caption
- Data is loaded into Ray Data for distributed multimodal processing
- We can easily scale this to millions of real multimodal samples

## Step 2: Image Feature Extraction
*⏱ Time: 8 minutes*

### What We're Doing
Extract meaningful features from images using a pre-trained computer vision model. These features will later be combined with text features for multimodal understanding.

### Why Image Features Matter
- **Semantic Understanding**: Convert raw pixels into meaningful representations
- **Efficiency**: Pre-trained models save time and computational resources
- **Compatibility**: Features can be easily combined with other modalities

In [ ]:
from torchvision import models, transforms
from transformers import AutoTokenizer, AutoModel

class ImageFeatureExtractor:
    """Extract features from images using a pre-trained ResNet model."""
    
    def __init__(self):
        print("📷 Loading image feature extraction model...")
        # Load pre-trained ResNet model (removing final classification layer)
        self.model = models.resnet50(pretrained=True)
        self.model = torch.nn.Sequential(*list(self.model.children())[:-1])  # Remove final layer
        self.model.to(device)
        self.model.eval()
        
        # Define image preprocessing transforms
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),  # ResNet expects 224x224 images
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        print(" Image model loaded successfully")
    
    def extract_features(self, batch):
        """Extract features from a batch of images."""
        images = batch["image"]
        
        # Preprocess images
        processed_images = []
        for img in images:
            try:
                # Convert numpy array to PIL Image and apply transforms
                processed_img = self.transform(img)
                processed_images.append(processed_img)
            except Exception as e:
                print(f" Error processing image: {e}")
                # Use zero tensor as fallback
                processed_images.append(torch.zeros(3, 224, 224))
        
        # Stack into batch tensor
        batch_tensor = torch.stack(processed_images).to(device)
        
        # Extract features using ResNet
        with torch.no_grad():
            features = self.model(batch_tensor)
            # Flatten features (batch_size, 2048) -> (batch_size, 2048)
            features = features.squeeze()
            
        return {
            **batch,  # Keep original data
            "image_features": features.cpu().numpy().tolist()  # Add extracted features
        }

# Apply image feature extraction to our dataset
print(" Extracting image features...")
start_time = time.time()

image_features_dataset = multimodal_dataset.map_batches(
    ImageFeatureExtractor,
    batch_size=32,  # Process 32 images at a time
    concurrency=1 if device == "cuda" else 2,  # Use single GPU worker or multiple CPU workers
    num_gpus=1 if device == "cuda" else 0
)

extraction_time = time.time() - start_time
print(f" Image feature extraction completed in {extraction_time:.2f} seconds")

# Validate the results
sample_features = image_features_dataset.take(1)[0]
print(f" Feature vector shape: {len(sample_features['image_features'])} dimensions")
print(f" Feature range: {min(sample_features['image_features']):.3f} to {max(sample_features['image_features']):.3f}")

** What's happening here?**
- **Feature Extraction**: ResNet converts each image into a 2048-dimensional feature vector
- **Preprocessing**: Images are resized and normalized to match the model's training data
- **Batch Processing**: Multiple images processed simultaneously for efficiency
- **Error Handling**: Graceful handling of any image processing issues

## Step 3: Text Feature Extraction
*⏱ Time: 8 minutes*

### What We're Doing
Extract semantic features from text using a pre-trained language model. These text embeddings capture the meaning of captions and can be combined with image features.

### Why Text Features Matter
- **Semantic Understanding**: Convert words into numerical representations that capture meaning
- **Cross-Modal Alignment**: Text and image features can be compared and combined
- **Scalability**: Process thousands of text descriptions efficiently

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

class TextFeatureExtractor:
    """Extract features from text using a pre-trained BERT model."""
    
    def __init__(self):
        print(" Loading text feature extraction model...")
        # Use a lightweight BERT model for text understanding
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.to(device)
        self.model.eval()
        print(" Text model loaded successfully")
    
    def extract_features(self, batch):
        """Extract features from a batch of text descriptions."""
        texts = batch["text"]
        
        try:
            # Tokenize all texts in the batch
            encoded = self.tokenizer(
                texts,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors="pt"
            ).to(device)
            
            # Extract features using BERT
            with torch.no_grad():
                outputs = self.model(**encoded)
                # Use mean pooling to get sentence-level features
                features = outputs.last_hidden_state.mean(dim=1)
                # Normalize features for better multimodal alignment
                features = F.normalize(features, p=2, dim=1)
            
            return {
                **batch,  # Keep original data
                "text_features": features.cpu().numpy().tolist()  # Add extracted features
            }
            
        except Exception as e:
            print(f" Error processing text batch: {e}")
            # Return batch with zero features as fallback
            batch_size = len(texts)
            feature_dim = 384  # MiniLM feature dimension
            zero_features = [[0.0] * feature_dim] * batch_size
            return {
                **batch,
                "text_features": zero_features
            }

# Apply text feature extraction to our dataset
print(" Extracting text features...")
start_time = time.time()

text_features_dataset = image_features_dataset.map_batches(
    TextFeatureExtractor,
    batch_size=64,  # Process more texts at once (they're smaller than images)
    concurrency=1 if device == "cuda" else 2,
    num_gpus=1 if device == "cuda" else 0
)

text_extraction_time = time.time() - start_time
print(f" Text feature extraction completed in {text_extraction_time:.2f} seconds")

# Validate the results
sample_text_features = text_features_dataset.take(1)[0]
print(f" Text feature vector shape: {len(sample_text_features['text_features'])} dimensions")
print(f" Text feature range: {min(sample_text_features['text_features']):.3f} to {max(sample_text_features['text_features']):.3f}")

# Show we now have both image and text features
print(f"\n Multimodal features ready:")
print(f"  - Image features: {len(sample_text_features['image_features'])} dimensions")
print(f"  - Text features: {len(sample_text_features['text_features'])} dimensions")
print(f"  - Total feature space: {len(sample_text_features['image_features']) + len(sample_text_features['text_features'])} dimensions")

** What's happening here?**
- **Text Encoding**: BERT converts text into 384-dimensional semantic vectors
- **Batch Processing**: Multiple text descriptions processed simultaneously
- **Feature Normalization**: Text features normalized for better multimodal alignment
- **Error Resilience**: Robust error handling for text processing issues

## Step 4: Cross-Modal Fusion
*⏱ Time: 7 minutes*

### What We're Doing
Combine image and text features to create unified multimodal representations. This is where the magic of multimodal AI happens - understanding content that combines visual and textual information.

### Why Fusion Matters
- **Richer Understanding**: Combined features capture more information than either modality alone
- **Better Predictions**: Multimodal models consistently outperform single-modal approaches
- **Real-World Relevance**: Most real content is multimodal (social posts, product listings, etc.)

In [ ]:
class MultimodalFusion:
    """Combine image and text features into unified representations."""
    
    def __init__(self):
        print(" Initializing multimodal fusion...")
        # We'll use simple concatenation and weighted fusion
        # In production, you might use attention mechanisms or learned fusion
        self.image_weight = 0.6  # Weight for image features
        self.text_weight = 0.4   # Weight for text features
        print(" Fusion weights configured")
    
    def fuse_features(self, batch):
        """Fuse image and text features into multimodal representations."""
        try:
            image_features = np.array(batch["image_features"])
            text_features = np.array(batch["text_features"])
            
            # Method 1: Simple concatenation
            concatenated_features = np.concatenate([image_features, text_features], axis=1)
            
            # Method 2: Weighted fusion (average of normalized features)
            # Normalize features to same scale for fair weighting
            image_norm = image_features / (np.linalg.norm(image_features, axis=1, keepdims=True) + 1e-8)
            text_norm = text_features / (np.linalg.norm(text_features, axis=1, keepdims=True) + 1e-8)
            
            weighted_features = (
                self.image_weight * image_norm + 
                self.text_weight * text_norm
            )
            
            # Calculate similarity between image and text features
            similarity_scores = []
            for img_feat, txt_feat in zip(image_norm, text_norm):
                # Cosine similarity between image and text features
                similarity = np.dot(img_feat, txt_feat) / (
                    np.linalg.norm(img_feat) * np.linalg.norm(txt_feat) + 1e-8
                )
                similarity_scores.append(float(similarity))
            
            return {
                **batch,  # Keep original data
                "multimodal_features_concat": concatenated_features.tolist(),
                "multimodal_features_weighted": weighted_features.tolist(),
                "image_text_similarity": similarity_scores
            }
            
        except Exception as e:
            print(f" Error in multimodal fusion: {e}")
            batch_size = len(batch["image_features"])
            # Return empty features as fallback
            return {
                **batch,
                "multimodal_features_concat": [[0.0] * 2432] * batch_size,  # 2048 + 384
                "multimodal_features_weighted": [[0.0] * 2048] * batch_size,
                "image_text_similarity": [0.0] * batch_size
            }

# Apply multimodal fusion to our dataset
print(" Performing multimodal fusion...")
start_time = time.time()

final_multimodal_dataset = text_features_dataset.map_batches(
    MultimodalFusion,
    batch_size=128,  # Larger batches for fusion operations
    concurrency=4,   # CPU-intensive operation, use multiple workers
    num_gpus=0       # Fusion doesn't need GPU
)

fusion_time = time.time() - start_time
print(f" Multimodal fusion completed in {fusion_time:.2f} seconds")

# Analyze the fused results
fusion_results = final_multimodal_dataset.take(5)

print("\n Multimodal Fusion Results:")
print("-" * 60)
for i, result in enumerate(fusion_results):
    similarity = result['image_text_similarity']
    text_preview = result['text'][:40]
    
    print(f"{i+1}. '{text_preview}...'")
    print(f"   Image-Text Similarity: {similarity:.3f}")
    print(f"   Concat Features: {len(result['multimodal_features_concat'])} dims")
    print(f"   Weighted Features: {len(result['multimodal_features_weighted'])} dims")
    print()

# Performance summary with profiling (rule #199: Include performance profiling)
total_samples = final_multimodal_dataset.count()
total_processing_time = time.time() - start_time

print(f" Final Results:")
print(f"  - Processed {total_samples:,} multimodal samples")
print(f"  - Total processing time: {total_processing_time:.2f} seconds")
print(f"  - Processing rate: {total_samples/total_processing_time:.1f} samples/second")
print(f"  - Created rich multimodal representations")
print(f"  - Ready for downstream AI tasks (classification, search, etc.)")

# Performance profiling summary
print(f"\n Performance Breakdown:")
print(f"  - Image feature extraction: {extraction_time:.2f}s")
print(f"  - Text feature extraction: {text_extraction_time:.2f}s") 
print(f"  - Multimodal fusion: {fusion_time:.2f}s")
print(f"  - Total pipeline time: {total_processing_time:.2f}s")

# Resource utilization summary
cluster_resources = ray.cluster_resources()
print(f"\n Resource Utilization:")
print(f"  - CPUs available: {cluster_resources.get('CPU', 0)}")
print(f"  - GPUs available: {cluster_resources.get('GPU', 0)}")
print(f"  - Memory available: {cluster_resources.get('memory', 0)/1e9:.1f} GB")

# Clean up resources
ray.shutdown()
print(" Ray cluster shut down successfully!")

** What we accomplished:**
- **Cross-Modal Understanding**: Combined visual and textual information
- **Feature Fusion**: Created unified representations from separate modalities
- **Similarity Analysis**: Measured how well images and text align
- **Scalable Processing**: Handled multimodal data efficiently with Ray Data

---

## Troubleshooting Common Issues

### **Problem: "CUDA out of memory with multimodal models"**
**Solution**:

In [ ]:
# Reduce batch sizes and use CPU for fusion
image_batch_size = 16  # Smaller for GPU-intensive image processing
text_batch_size = 32   # Larger for CPU text processing
fusion_num_gpus = 0    # Use CPU for fusion operations

### **Problem: "Feature dimensions don't match for fusion"**
**Solution**:

In [ ]:
# Add dimension checking and padding
def align_feature_dimensions(feat1, feat2, target_dim=512):
    # Pad or truncate features to same dimension
    feat1_aligned = feat1[:target_dim] if len(feat1) > target_dim else feat1 + [0] * (target_dim - len(feat1))
    feat2_aligned = feat2[:target_dim] if len(feat2) > target_dim else feat2 + [0] * (target_dim - len(feat2))
    return feat1_aligned, feat2_aligned

### **Problem: "Low similarity scores between modalities"**
**Solution**:

In [ ]:
# Use better pre-trained models or fine-tune for your domain
# Consider using CLIP models that are trained for image-text alignment

### **Performance Optimization Tips**

1. **Model Selection**: Use CLIP models for better image-text alignment
2. **Batch Sizing**: Optimize batch sizes for each modality separately
3. **GPU Memory**: Process images on GPU, text on CPU if memory is limited
4. **Feature Caching**: Cache extracted features to avoid recomputation
5. **Fusion Methods**: Experiment with different fusion techniques

### **Performance Considerations**

Ray Data provides several advantages for multimodal processing:
- **Parallel modality processing**: Image and text features can be extracted simultaneously
- **GPU utilization**: Automatic distribution of GPU-intensive tasks across available hardware
- **Memory efficiency**: Large multimodal datasets are processed in manageable batches
- **Resource optimization**: Different modalities can use different resource configurations

---

## Next Steps and Extensions

### **Try These Advanced Features**
1. **CLIP Integration**: Use OpenAI's CLIP for better image-text understanding
2. **Attention Mechanisms**: Implement cross-modal attention for better fusion
3. **Multimodal Classification**: Build classifiers using the fused features
4. **Similarity Search**: Create image-text search and recommendation systems
5. **Real Datasets**: Use actual social media or e-commerce multimodal data

### **Production Considerations**
- **Model Optimization**: Use quantization and pruning for faster inference
- **Caching Strategy**: Cache frequently used features and models
- **Error Handling**: Implement robust error handling for production workloads
- **Monitoring**: Track model performance and feature quality
- **Scaling**: Use Ray Serve for real-time multimodal inference

### **Related Ray Data Templates**
- **Ray Data Batch Inference Optimization**: Optimize multimodal model inference
- **Ray Data NLP Text Analytics**: Deep dive into text processing techniques
- **Ray Data Batch Classification**: Focus on image processing optimization

** Congratulations!** You've successfully built a scalable multimodal AI pipeline with Ray Data!

These multimodal techniques enable you to build AI systems that understand content the way humans do - by combining visual and textual information for richer understanding.

## Architecture

```
S3 Data Sources → Ray Data → Modality-Specific Processing → Cross-Modal Fusion → AI Analysis → Results
     ↓              ↓              ↓                        ↓              ↓         ↓
  Images         Parallel      Vision Models            Embedding      ML Models  Insights
  Text           Processing    Text Encoders            Fusion         Inference  Reports
  Audio          GPU Workers   Audio Models             Aggregation    Scoring    Analytics
```

## Key Components

### 1. **Multimodal Data Loading**
- `ray.data.read_images()` for image data from S3
- `ray.data.read_text()` for text documents and social media posts
- `ray.data.read_binary_files()` for audio files
- Automatic format detection and validation

### 2. **Modality-Specific Processing**
- **Vision**: Pre-trained vision transformers (ViT, ResNet) with GPU acceleration
- **Text**: BERT, RoBERTa, or custom text encoders
- **Audio**: Wav2Vec, HuBERT, or audio feature extractors
- Parallel processing with device-specific optimizations

### 3. **Cross-Modal Fusion**
- Embedding alignment and normalization
- Attention mechanisms for cross-modal understanding
- Multimodal transformer architectures
- Feature concatenation and aggregation strategies

### 4. **AI Analysis and Inference**
- Multimodal classification models
- Content recommendation systems
- Sentiment and content analysis
- Automated content moderation

## Prerequisites

- Ray cluster with GPU support (recommended)
- Python 3.8+ with required ML libraries
- Access to S3 or local multimodal datasets
- Basic understanding of computer vision, NLP, and audio processing

## Installation

In [ ]:
%%bash
pip install ray[data] torch torchvision transformers
pip install torchaudio librosa pillow opencv-python
pip install sentence-transformers accelerate

## 5-Minute Quick Start

**Goal**: Get a multimodal AI pipeline running in 5 minutes with real data

### **Step 1: Setup on Anyscale (30 seconds)**

In [ ]:
# Ray cluster is already running on Anyscale
import ray

# Check cluster status (already connected)
print('Connected to Anyscale Ray cluster!')
print(f'Available resources: {ray.cluster_resources()}')

# Install any missing packages if needed
# !pip install torch torchvision transformers

### **Step 2: Load Real Data (1 minute)**

In [ ]:
from ray.data import read_images

# Load real ImageNet subset (Imagenette) - publicly available
image_ds = read_images("s3://anonymous@air-example-data-2/imagenette2/train/", mode="RGB").limit(10)
print(f"Loaded {image_ds.count()} real images")

# Quick data inspection
sample = image_ds.take(1)[0]
print(f"Image shape: {sample['image'].shape}")

### **Step 3: Run Vision Processing (2 minutes)**

In [ ]:
import torch
from torchvision import models, transforms

class QuickVisionProcessor:
    def __init__(self):
        self.model = models.resnet18(pretrained=True)  # Smaller model for speed
        self.model.eval()
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __call__(self, batch):
        features = []
        for img in batch["image"]:
            try:
                img_tensor = self.transform(img).unsqueeze(0)
                with torch.no_grad():
                    feature = self.model(img_tensor)
                features.append(feature.numpy()[0])
            except Exception as e:
                print(f"Error processing image: {e}")
                features.append(None)
        return {"features": features}

# Process images
processed = image_ds.map_batches(QuickVisionProcessor(), batch_size=4)
results = processed.take(5)
print(f"Processed {len(results)} image batches")

### **Step 4: View Results (1 minute)**

In [ ]:
# Display results
for i, result in enumerate(results):
    features = result.get("features", [])
    valid_features = [f for f in features if f is not None]
    print(f"Batch {i}: {len(valid_features)} successful feature extractions")

print("Quick start completed! Check the full demo for advanced features.")

## Complete Tutorial

### 1. **Load Real Multimodal Data**

In [ ]:
import ray
from ray.data import read_images, read_text, read_binary_files

# Initialize Ray
ray.init()

# Load real datasets from public sources
# ImageNet subset (Imagenette) - publicly available
image_ds = read_images("s3://anonymous@air-example-data-2/imagenette2/train/", mode="RGB")

# Common Crawl news articles - publicly available
text_ds = read_text("s3://anonymous@commoncrawl/crawl-data/CC-NEWS/2023/01/")

# LibriSpeech audio dataset - publicly available  
audio_ds = read_binary_files("s3://anonymous@openslr/12/train-clean-100/")

print(f"Images (Imagenette): {image_ds.count()}")
print(f"Text (Common Crawl): {text_ds.count()}")
print(f"Audio (LibriSpeech): {audio_ds.count()}")

### 2. **Process Each Modality**

In [ ]:
from ray.data import ActorPoolStrategy
import torch

class VisionProcessor:
    def __init__(self):
        self.model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
        self.model.eval()
        if torch.cuda.is_available():
            self.model.cuda()
    
    def __call__(self, batch):
        # Process image batch
        images = torch.stack([torch.from_numpy(img) for img in batch["image"]])
        if torch.cuda.is_available():
            images = images.cuda()
        
        with torch.no_grad():
            features = self.model(images)
        
        return {"image_features": features.cpu().numpy()}

# Apply vision processing
processed_images = image_ds.map_batches(
    VisionProcessor,
    batch_size=32,
    num_gpus=1,
    concurrency=4
)

### 3. **Cross-Modal Fusion**

In [ ]:
def fuse_modalities(batch):
    """Combine features from different modalities"""
    # Align features by content ID
    image_features = batch["image_features"]
    text_features = batch["text_features"]
    audio_features = batch["audio_features"]
    
    # Simple concatenation (can be enhanced with attention)
    fused_features = np.concatenate([
        image_features, text_features, audio_features
    ], axis=1)
    
    return {"fused_features": fused_features}

# Apply fusion (assuming aligned datasets)
fused_ds = processed_images.map_batches(fuse_modalities)

### 4. **Multimodal Analysis**

In [ ]:
class MultimodalClassifier:
    def __init__(self):
        self.classifier = torch.nn.Linear(2048 + 768 + 512, 10)  # Example dimensions
        if torch.cuda.is_available():
            self.classifier.cuda()
    
    def __call__(self, batch):
        features = torch.from_numpy(batch["fused_features"]).float()
        if torch.cuda.is_available():
            features = features.cuda()
        
        with torch.no_grad():
            predictions = self.classifier(features)
            probabilities = torch.softmax(predictions, dim=1)
        
        return {
            "predictions": predictions.cpu().numpy(),
            "probabilities": probabilities.cpu().numpy()
        }

# Apply classification
results = fused_ds.map_batches(
    MultimodalClassifier,
    batch_size=64,
    num_gpus=1,
    concurrency=2
)

## Advanced Features

### **GPU Memory Management**
- Automatic batch size optimization based on GPU memory
- Gradient checkpointing for large models
- Memory-efficient data loading and processing

### **Scalability**
- Horizontal scaling across multiple GPUs
- Automatic load balancing and resource allocation
- Support for heterogeneous GPU clusters

### **Performance Optimization**
- Operator fusion for reduced memory transfers
- Pipelined processing for continuous data flow
- Caching strategies for frequently accessed data

## Production Considerations

### **Model Serving**
- Integration with Ray Serve for real-time inference
- Model versioning and A/B testing
- Automatic scaling based on demand

### **Monitoring and Observability**
- Performance metrics and resource utilization
- Error tracking and alerting
- Pipeline health monitoring

### **Data Quality and Validation**
- Input validation and sanitization
- Output quality checks and confidence scoring
- Fallback strategies for failed processing

## Example Workflows

### **Content Moderation Pipeline**
1. Load social media content (images, text, audio)
2. Extract features using modality-specific models
3. Apply content moderation rules and ML models
4. Generate moderation decisions and confidence scores
5. Store results for audit and compliance

### **Recommendation System**
1. Process user interaction data (clicks, views, listens)
2. Generate user and content embeddings
3. Calculate similarity scores across modalities
4. Rank and recommend relevant content
5. Update recommendations in real-time

### **Market Intelligence**
1. Collect market data (news, social media, financial reports)
2. Extract sentiment and key information
3. Correlate across different data sources
4. Generate market insights and predictions
5. Alert on significant events or trends

## Performance Analysis

### **Benchmark Framework**

The template includes comprehensive performance measurement tools:

| Benchmark Type | Measurement Focus | Output Visualization |
|---------------|-------------------|---------------------|
| **Fusion Method Comparison** | Attention vs Weighted vs Simple | Performance comparison charts |
| **Batch Size Optimization** | Memory usage vs throughput | Optimization curves |
| **GPU vs CPU Analysis** | Device performance comparison | Speedup analysis |
| **Scalability Testing** | Multi-GPU performance | Scaling visualizations |

### **Performance Measurement Example**

In [ ]:
# Run actual performance benchmarks
benchmark = MultimodalBenchmark()
results = benchmark.run_performance_benchmark(
    fusion_methods=["simple", "weighted", "attention"],
    batch_sizes=[4, 8, 16, 32],
    gpu_enabled=True
)

# Generate verified performance report
benchmark.generate_benchmark_report()

# Expected output structure:
# - benchmark_results.csv: Detailed metrics
# - performance_report.txt: Analysis summary
# - performance_charts.html: Interactive visualizations

### **Modality Processing Pipeline**

```
Input Data Flow:
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   Images    │    │    Text     │    │   Audio     │
│ (ImageNet)  │    │   (IMDB)    │    │(LibriSpeech)│
└──────┬──────┘    └──────┬──────┘    └──────┬──────┘
       │                  │                  │
       ▼                  ▼                  ▼
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   ResNet    │    │    BERT     │    │   Audio     │
│ Features    │    │ Embeddings  │    │ Features    │
│ (2048-dim)  │    │ (384-dim)   │    │ (13-dim)    │
└──────┬──────┘    └──────┬──────┘    └──────┬──────┘
       │                  │                  │
       └──────────────────┼──────────────────┘
                          ▼
                ┌─────────────────┐
                │ Multimodal      │
                │ Fusion          │
                │ (Attention)     │
                └─────────┬───────┘
                          ▼
                ┌─────────────────┐
                │ Classification  │
                │ & Analysis      │
                └─────────────────┘
```

### **Expected Resource Requirements**

| Processing Stage | CPU Cores | Memory (GB) | GPU Memory (GB) | Processing Time |
|-----------------|-----------|-------------|-----------------|-----------------|
| **Image Processing** | 4-8 | 8-16 | 4-8 | Measured in demo |
| **Text Processing** | 2-4 | 4-8 | 2-4 | Measured in demo |
| **Audio Processing** | 2-4 | 4-8 | 1-2 | Measured in demo |
| **Fusion & Classification** | 4-8 | 8-16 | 4-8 | Measured in demo |

## Troubleshooting

### **Common Issues and Solutions**

| Issue | Symptoms | Solution | Prevention |
|-------|----------|----------|------------|
| **GPU Memory Errors** | `RuntimeError: CUDA out of memory` | Reduce batch size to 4-8, use CPU fallback | Monitor GPU memory usage, start with small batches |
| **Data Alignment Issues** | Mismatched modality counts | Ensure consistent IDs across datasets | Validate data alignment before processing |
| **Model Loading Failures** | Import errors, missing dependencies | Install required packages, check model availability | Use requirements.txt, test imports |
| **Poor Performance** | Slow processing, low GPU utilization | Optimize batch size, increase concurrency | Profile operations, monitor resource usage |
| **Memory Pressure** | Ray object store full | Reduce data in memory, process in chunks | Monitor object store, use streaming patterns |

### **Performance Optimization Guide**

In [ ]:
# Optimal configuration for different cluster types
import ray

# Check available resources
resources = ray.cluster_resources()
gpu_count = resources.get('GPU', 0)
cpu_count = resources.get('CPU', 0)

# Adjust configuration based on resources
if gpu_count >= 4:
    # Multi-GPU configuration
    batch_size = 32
    concurrency = 4
    num_gpus = 1
elif gpu_count >= 1:
    # Single GPU configuration
    batch_size = 16
    concurrency = 2
    num_gpus = 1
else:
    # CPU-only configuration
    batch_size = 8
    concurrency = cpu_count // 4
    num_gpus = 0

print(f"Recommended config: batch_size={batch_size}, concurrency={concurrency}, num_gpus={num_gpus}")

### **Debug Mode and Monitoring**

In [ ]:
import logging
import torch

# Enable comprehensive debugging
logging.basicConfig(level=logging.DEBUG)

# Monitor GPU memory if available
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1e9:.1f}GB")

# Enable Ray Data debugging
from ray.data.context import DataContext
ctx = DataContext.get_current()
ctx.enable_progress_bars = True

# Monitor processing with custom logging
class DebugProcessor:
    def __call__(self, batch):
        print(f"Processing batch with {len(batch)} items")
        print(f"GPU memory before: {torch.cuda.memory_allocated(0) / 1e9:.1f}GB" if torch.cuda.is_available() else "CPU mode")
        
        # Your processing logic here
        results = process_batch(batch)
        
        print(f"GPU memory after: {torch.cuda.memory_allocated(0) / 1e9:.1f}GB" if torch.cuda.is_available() else "Processing complete")
        return results

### **Error Recovery Strategies**

In [ ]:
# Implement robust error handling with recovery
def robust_multimodal_processing(batch):
    """Process multimodal data with comprehensive error recovery."""
    results = []
    errors = []
    
    for item in batch:
        try:
            # Attempt processing
            result = process_item(item)
            results.append(result)
            
        except torch.cuda.OutOfMemoryError:
            # GPU memory error - try CPU fallback
            try:
                torch.cuda.empty_cache()
                result = process_item_cpu(item)
                results.append(result)
                
            except Exception as fallback_error:
                errors.append(f"CPU fallback failed: {fallback_error}")
                results.append(create_error_result(item, fallback_error))
                
        except Exception as general_error:
            errors.append(f"Processing failed: {general_error}")
            results.append(create_error_result(item, general_error))
    
    if errors:
        print(f"Encountered {len(errors)} errors during processing")
    
    return results

## Next Steps

1. **Customize Models**: Replace pre-trained models with your own
2. **Add Modalities**: Extend to video, 3D data, or other formats
3. **Optimize Performance**: Tune batch sizes and resource allocation
4. **Scale Production**: Deploy to multi-GPU clusters with Ray Serve

## Resources

- [Ray Data Documentation](https://docs.ray.io/en/latest/data/index.html)
- [PyTorch Multimodal Tutorials](https://pytorch.org/tutorials/)
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/)
- [Ray GPU Support](https://docs.ray.io/en/latest/ray-core/using-ray-with-gpus.html)

---

*This template provides a foundation for building production-ready multimodal AI pipelines with Ray Data. Start with the basic examples and gradually add complexity based on your specific use case and requirements.*